<a href="https://colab.research.google.com/github/JackGraymer/Advanced-GenAI/blob/stage1-rerun/2.1_rag_research_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Generative Artificial Intelligence
**Project - Designing a RAG-Based Q&A System for News Retrieval**

**Authors:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan (Group 5)


# Step 2.1 - Building the RAG System: Research Agents -<br> Implementing multiple pre-retrieval and retrieval strategies

**Contribution:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan

**Goal of this step:** Implementing and comparing diﬀerent pre-retrieval and retrieval strategies to extract relevant candidate documents. These retrieved documents will go through the post-retrieval process in Step 2.2.

#  1. Data Preprocessing and Benchmark Construction

## 1.1 Setup of the environment

### Installations and loading of packages

Below the necessary libraries are installed and loaded into the environment.

In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import tempfile

### Setting seeds and mounting Google Drive storage folder

In [2]:
# Set the seed for consistent results
seed_value = 2138247234
random.seed(seed_value)
np.random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

Below we mount a shared Google Drive folder as a data storage and define the base path of the folder that will be used in the runtime.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_folder = '/content/drive/MyDrive/AdvGenAI'

### Check properties of environment

In [5]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
!pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.6.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.6
ale-py                                0.11.0
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.21.0
astropy                               7.0.2
astropy-iers-data                     0.2025.5.12.0.38.29
astunparse                            1

## 1.2 Loading the dataset from stage 1

In [7]:
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-08-final.csv'))

In [9]:
df.to_csv(os.path.join(base_folder, 'Stage1/Final-Datasets/Stage1-final.csv'), index=False)

In [10]:
df.head()

,folder_path,file_name,full_path,html_content,bs_html_title,markdown_content_hybrid,year,month,language,type,html_title,content,named_entities,topics,keywords,summary
0,/content/drive/MyDrive/AdvGenAI/data/de_intern...,der-r-pionier1.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p>Währe...",NaN,Während andere in den Sechziger-Jahren Lokführ...,2014,1,de,internal,Der r pionier1,Während andere in den Sechziger-Jahren Lokführ...,"[('Irene Urbanetti', 'PER'), ('Paul Studerus',...","['institut', 'statistik', 'abteilung', 'depart...","['computerprogrammen', 'technologie', 'statist...","Als die Studienentscheidung näher rückte, war ..."
1,/content/drive/MyDrive/AdvGenAI/data/de_intern...,web-of-science-alles-neu-macht-der--januar-.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p><a cl...",NaN,Weitere Informationen\n\n## Staffnet\n\nDas In...,2014,1,de,internal,Web of science alles neu macht der januar,Weitere Informationen ## Staffnet Das Info-Por...,"[('Info-Portal für Mitarbeitende', 'ORG'), ('E...","['informationen', 'newsletter', 'intern', 'abo...","['informationen', 'newsletter', 'mitarbeitende...",Weitere Informationen ## Staffnet Das Info-Por...
2,/content/drive/MyDrive/AdvGenAI/data/de_intern...,swiss-life-sciences-2014-experten-gesucht.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<h2>Staf...",NaN,## Staffnet\n\nDas Info-Portal für Mitarbeiten...,2014,1,de,internal,Swiss life sciences 2014 experten gesucht,## Staffnet Das Info-Portal für Mitarbeitende ...,"[('Info-Portal für Mitarbeitende', 'ORG'), ('E...","['newsletter', 'wichtigsten', 'abonnieren', 'a...","['newsletter', 'informationen', 'staffnet', 'a...",## Staffnet Das Info-Portal für Mitarbeitende ...
3,/content/drive/MyDrive/AdvGenAI/data/de_intern...,mendeley-literaturverwaltung.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p><a cl...",NaN,Weitere Informationen\n\n## Staffnet\n\nDas In...,2014,1,de,internal,Mendeley literaturverwaltung,Weitere Informationen ## Staffnet Das Info-Por...,"[('Info-Portal für Mitarbeitende', 'ORG'), ('E...","['informationen', 'newsletter', 'intern', 'sta...","['informationen', 'newsletter', 'mitarbeitende...",Weitere Informationen ## Staffnet Das Info-Por...
4,/content/drive/MyDrive/AdvGenAI/data/de_intern...,mehr-orientierung.html,/content/drive/MyDrive/AdvGenAI/data/de_intern...,"<div class=""text-image cq-dd-image"">\n<p>Das S...",NaN,Das Studierendenleben an der ETH dreht sich mo...,2014,1,de,internal,Mehr orientierung,Das Studierendenleben an der ETH dreht sich mo...,"[('Bergen', 'LOC'), ('Uster', 'LOC'), ('WiNaFe...","['eth', 'studierende', 'hochschule', 'anstatt'...","['prüfungen', 'prüfungssession', 'prüfungsbegi...",Während andere sich in den Bergen beim Skifahr...


## 1.3 Chunking of the news texts

In [29]:
import re
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

class HierarchicalRecursiveTextSplitter:
    def __init__(self, min_chunk_size=200, max_chunk_size=800, chunk_overlap=100):
        self.min_chunk_size = min_chunk_size
        self.max_chunk_size = max_chunk_size
        self.chunk_overlap = chunk_overlap
        self.recursive_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_size=max_chunk_size,
            chunk_overlap=chunk_overlap,
        )

    def split_by_header(self, text: str) -> List[str]:
        # Use multiline matching to split on header lines starting with '## '
        pattern = re.compile(r'(^## .+$)', flags=re.MULTILINE)
        splits = pattern.split(text)

        chunks = []
        if splits[0].strip():
            chunks.append(splits[0].strip())
        # Now iterate in step 2: header at i, content i+1
        for i in range(1, len(splits), 2):
            header = splits[i].strip()
            content = splits[i + 1].strip() if (i + 1) < len(splits) else ""
            chunk_text = f"{header}\n{content}"
            chunks.append(chunk_text)
        return chunks

    def merge_small_chunks(self, chunks: List[str]) -> List[str]:
        merged_chunks = []
        for chunk in chunks:
            if len(chunk) < self.min_chunk_size:
                if merged_chunks:
                    # Append small chunk to previous chunk with a space separator
                    merged_chunks[-1] += "\n\n" + chunk  # add double newline to separate naturally
                else:
                    # No previous chunk, start new one with this small chunk
                    merged_chunks.append(chunk)
            else:
                # Chunk big enough, start a new chunk
                merged_chunks.append(chunk)
        return merged_chunks

    def recursive_split(self, chunks: List[str]) -> List[str]:
        # Use recursive splitter to chunk large pieces
        final_chunks = []
        for chunk in chunks:
            if len(chunk) > self.max_chunk_size:
                # recursively split
                smaller_chunks = self.recursive_splitter.split_text(chunk)
                final_chunks.extend(smaller_chunks)
            else:
                final_chunks.append(chunk)
        return final_chunks

    def split_text(self, text: str) -> List[str]:
        # 1. Split hierarchically by header
        chunks = self.split_by_header(text)
        # 2. Merge too small chunks
        chunks = self.merge_small_chunks(chunks)
        # 3. Recursively split too large chunks
        chunks = self.recursive_split(chunks)
        return chunks

In [36]:
text = """
Dass sie mit 50 Jahren einmal die Assistentin des Leiters Human Resources werden würde, hätte sich Nicole Kubli als junge Frau nie gedacht. Vor 30 Jahren war sie frisch gebackene medizinische Praxisangestellte. Anders als ihre Berufskolleginnen, die in einer Arztpraxis unterkamen, begann ihr Arbeitsleben am Institut für Verhaltenswissenschaften der ETH. «Für mich war das eine ganz neue Welt.» Vor allem wegen der spürbaren Leidenschaft der Forscher, wie sie sagt. «Ich hatte davor nur wenige Leute kennen gelernt, die so für ihre Berufung lebten.»

Neun Jahre unterstütze sie dort die Forschung bei komplexen psychophysiologischen Humanversuchen, dann ging der Professor in Pension. Anders als die meisten ETH-Angestellten, die nach einer Emeritierung des Chefs weiterhin im Job verbleiben, wollte die damals 29-Jährige Neues ausprobieren. Einige Monate später kam sie als Quereinsteigerin ins Sekretariat des Laboratoriums für Festkörperphysik. «Mir gefiel, dass dort so viele Leute aus der ganzen Welt forschten.» Zumal gerade diese auch mit ganz profanen Anliegen zu ihr kamen: Mit der Frage wo man die löchrigen Hosen flicken lassen konnte, zum Beispiel. Genau das habe sie oft zum Schmunzeln gebracht: «Ich genoss es, dass die Atmosphäre so ungezwungen war.» Nicht für lange. Auch an diesem Arbeitsplatz bahnte sich 1999 ein Chefwechsel aufgrund einer Emeritierung an. Wieder räumte sie daraufhin freiwillig ihr Pult.

«Diesmal wollte ich eine Stelle mit einem jüngeren Chef, der nicht in absehbarer Zeit in Pension gehen würde», erinnert sie sich. Bei der Personalabteilung wurde sie fündig. Von der früheren Arbeit an der Forscherfront profitiert sie in der heutigen Position. «Ich verstehe beide Seiten sehr gut – die Forschenden und die Administration.» Gibt ein Professor eine Rechnung bei ihr unvollständig ausgefüllt ab, nimmt Kubli dies gelassen, wie sie sagt. «Wissenschaftler konzentrieren sich auf ihre Projekte, nicht auf Papiere.»

## Vom Bürosessel in den Pferdesattel

Nicole Kubli war nie nur in der Administration zuhause. Ihre grosse Leidenschaft gilt den Pferden und dem Western-Reitsport. Jeden Tag nach der Arbeit und am Wochenende reitet sie mit ihrem Mann aus. 2004 machte sie ihre Leidenschaft mit einer Ausbildung zur diplomierten Reittherapeutin gar zum Teilzeit-Beruf. Jeden Freitag arbeitet sie nun mit behinderten Kindern eines Sonderschulheims. Laut Kubli haben die leistungsorientierte ETH und die sozialorientierte Sonderschule einiges gemeinsam: Der Forscher müsse seine Ergebnisse in renommierten Fachzeitschriften publizieren können, ein Rollstuhlfahrer müsse es schaffen, sich aufs Pferd hinaufzuziehen. «Beide müssen im Rahmen ihrer Möglichkeiten Spitzenleistungen erbringen.»

Gerade steht wieder ein Umbruch an: der dritte Chefwechsel. «Wie es für mich weitergeht, steht noch in den Sternen.»

## Dienstjubiläen Oktober 2014

35 Jahre
 Prof. Dr. Neil Mancktelow, Geologisches Institut

30 Jahre
 Dr. Emmanuel Baltsavias, Inst. f. Geodäsie u. Photogrammetrie
 Prof. Dr. Stefano Bernasconi, Geologisches Institut
 Nicole Kubli, Human Resources

25 Jahre
 Ruth Alder Broens, Inst. f. Pharmazeutische Wissenschaften
 Thomas Lutz, Personal und Dienste
 Margrit Oertle, GS Geotechnische Kommission
 Erich Scherer, ETH-Bibliothek
 Barbara Schröder Würtz Personalentwicklung

20 Jahre
 Sandro Bösch-Pauli, Institut für Umweltentscheidungen
 Dr. Thomas Uli Burger, Institut für Integrierte Systeme
 Ruth Kühne, Institut für Agrarwissenschaften
 Rainer Schwab, Betrieb
 Rudolf Streuli, Betrieb

15 Jahre
 Enrico Fiorentino Manna, Studiensekretariat Bauingenieurwissenschaften
 Dr. Thomas Tschan, Institut für Biochemie
 Annette Walzer, Institut f. Bau-und Infrastrukturmanagement

10 Jahre
 Dr. Emine Elvan Kut Bacs, Collegium Helveticum
 Chrissula Chatzidis, Inst. f. Pharmazeutische Wissenschaften
 Roland Jürg Lüthi, ETH-Bibliothek
 Dr. Györgi Zoltan Nagy, Professur Architektur und Geb.tech.
 Milena Pfister, ETH-Bibliothek

Altersrücktritte
 Walter Aschwanden, Fachbereich GA
 Rolf Kägi, Hörsaalbetrieb
 Dr. Giuseppe Gabriele Giovanni Manzardo, Inst. f. Lebensmittelwiss.
 Prof. Dr. Helmut Jürg Weissert, Geologisches Institut
 Max Wohlwend, Inst. f. Chemie- und Bio-Ingenieurwissenschaften

Todesfälle
 Frau Olympia Stefaní, EU GrantsAccess

## Keine News verpassen

Abonnieren Sie den internen Newsletter

## Staffnet

Das Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.

## Newsletter abonnieren

Für Newsletter «Intern aktuell» anmelden
"""

splitter = HierarchicalRecursiveTextSplitter(
    min_chunk_size=200,
    max_chunk_size=800,
    chunk_overlap=100,
)

chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk)
    print()

--- Chunk 1 ---
Dass sie mit 50 Jahren einmal die Assistentin des Leiters Human Resources werden würde, hätte sich Nicole Kubli als junge Frau nie gedacht. Vor 30 Jahren war sie frisch gebackene medizinische Praxisangestellte. Anders als ihre Berufskolleginnen, die in einer Arztpraxis unterkamen, begann ihr Arbeitsleben am Institut für Verhaltenswissenschaften der ETH. «Für mich war das eine ganz neue Welt.» Vor allem wegen der spürbaren Leidenschaft der Forscher, wie sie sagt. «Ich hatte davor nur wenige Leute kennen gelernt, die so für ihre Berufung lebten.»

--- Chunk 2 ---
Neun Jahre unterstütze sie dort die Forschung bei komplexen psychophysiologischen Humanversuchen, dann ging der Professor in Pension. Anders als die meisten ETH-Angestellten, die nach einer Emeritierung des Chefs weiterhin im Job verbleiben, wollte die damals 29-Jährige Neues ausprobieren. Einige Monate später kam sie als Quereinsteigerin ins Sekretariat des Laboratoriums für Festkörperphysik. «Mir gefiel, dass d